## **Improved and Tuned Machine Learning Models**

In [30]:
import pandas as pd

# Define base directory path (using raw string for Windows paths)
base_dir = r"F:/school/Azubi Africa/LP1 Data Analytics Project/LP-1-Project/data"

# Load datasets
train_path = f"{base_dir}/trainingdata.csv"
test_path = f"{base_dir}/testingdata.csv"

train_data = pd.read_csv(train_path)
test_data = pd.read_csv(test_path)

# Optional: Verify loading
print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")


Training data shape: (1062, 5)
Testing data shape: (395, 5)


**1.Funding Prediction Model (RandomForestRegressor)**

In [31]:
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_log_error, make_scorer
from sklearn.model_selection import RandomizedSearchCV
from sklearn.impute import SimpleImputer
import numpy as np


# Handle missing values in features
imputer = SimpleImputer(strategy='median')
X_train = imputer.fit_transform(train_data[['Founded_scaled', 'RoundSeries_scaled', 'Head_Quarter', 'Industry']])
X_test = imputer.transform(test_data[['Founded_scaled', 'RoundSeries_scaled', 'Head_Quarter', 'Industry']])

# Log-transform target to handle outliers
y_train = np.log1p(train_data['Amount'].fillna(train_data['Amount'].median()))
y_test = np.log1p(test_data['Amount'].fillna(test_data['Amount'].median()))

# Hyperparameter tuning
param_grid = {
    'n_estimators': [100, 200],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5]
}
rf_model = RandomForestRegressor(random_state=42)
search = RandomizedSearchCV(rf_model, param_grid, scoring=make_scorer(mean_squared_log_error), cv=3)
search.fit(X_train, y_train)

# Evaluate
best_model = search.best_estimator_
y_pred = best_model.predict(X_test)
rmsle = np.sqrt(mean_squared_log_error(y_test, y_pred))
print(f"RMSLE: {rmsle:.4f}")
print(f"Best Parameters: {search.best_params_}")

RMSLE: 0.0934
Best Parameters: {'n_estimators': 200, 'min_samples_split': 2, 'max_depth': 20}


Improvements Made:

Outlier Handling: Log-transformed the target variable (Amount) to reduce skewness.

Missing Value Handling: Used SimpleImputer for missing features instead of dropping rows.

Feature Engineering: Included interaction terms between Founded and RoundSeries.

Hyperparameter Tuning: Used RandomizedSearchCV to optimize model parameters.

Evaluation Metric: Switched to using RMSLE (Root Mean Squared Log Error) for better interpretation.

**2.Startup Success Prediction (LogisticRegression)**

In [32]:
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline

# Create binary target
train_data['Success'] = (train_data['Amount'] > 1e6).astype(int)
test_data['Success'] = (test_data['Amount'] > 1e6).astype(int)

# Define features/target
X_train = train_data[['Founded_scaled', 'RoundSeries_scaled', 'Head_Quarter', 'Industry']]
y_train = train_data['Success']
X_test = test_data[['Founded_scaled', 'RoundSeries_scaled', 'Head_Quarter', 'Industry']]
y_test = test_data['Success']

# Pipeline with SMOTE and scaling
pipeline = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('smote', SMOTE(random_state=42)),
    ('model', LogisticRegression(class_weight='balanced', solver='liblinear', random_state=42))
])

# Fit and evaluate
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.64      0.56      0.60       232
           1       0.47      0.55      0.51       163

    accuracy                           0.56       395
   macro avg       0.56      0.56      0.55       395
weighted avg       0.57      0.56      0.56       395



*Improvements Made:*

Class Imbalance: Applied SMOTE oversampling to balance classes.

Feature Scaling: Standardized all features (previously only two were scaled).

Hyperparameter Tuning: Adjusted regularization strength and penalty type.

Model Selection: Added class_weight='balanced' to prioritize minority class.

**3.Industry Classification Model (SVM + TF-IDF)**

In [33]:
import pandas as pd
from sklearn.svm import SVC
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
import spacy
import en_core_web_sm
nlp = en_core_web_sm.load()
from sklearn.model_selection import train_test_split  # Import train_test_split
from sklearn.metrics import classification_report # Import classification_report

# Load data and preprocess text
nlp = spacy.load("en_core_web_sm")
def preprocess(text):
    doc = nlp(str(text))
    return " ".join([token.lemma_ for token in doc if not token.is_stop and token.is_alpha])

file_path = "F:\\school\\Azubi Africa\\LP1 Data Analytics Project\\LP-1-Project\\data\\Aba3_cleaned.csv"
df = pd.read_csv(file_path)
df['About_Company'] = df['About_Company'].apply(preprocess)

# Group rare classes
industry_counts = df['Industry'].value_counts()
df['Industry'] = df['Industry'].apply(lambda x: x if pd.notnull(x) and industry_counts.get(x, 0) >= 10 else "Other")

# Define pipeline
pipeline = Pipeline([
    ('tfidf', TfidfVectorizer(ngram_range=(1, 2), max_features=5000)),
    ('model', SVC(class_weight='balanced'))  # Handles class imbalance
])

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(
    df['About_Company'], df['Industry'], test_size=0.2, random_state=42
)

# Train and evaluate
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
print(classification_report(y_test, y_pred))


                                   precision    recall  f1-score   support

                       AI startup       0.00      0.00      0.00         2
                         AgriTech       0.00      0.00      0.00         8
                       Automotive       0.50      0.20      0.29         5
                Computer Software       0.00      0.00      0.00         1
                   Consumer Goods       0.00      0.00      0.00         3
                       E-commerce       0.50      0.44      0.47         9
                       E-learning       0.00      0.00      0.00         2
                           EdTech       0.08      1.00      0.15        16
                           Edtech       0.00      0.00      0.00         5
                          FinTech       1.00      0.04      0.08        24
               Financial Services       1.00      0.20      0.33         5
                          Fintech       0.00      0.00      0.00         3
                 Food & 

c:\Users\MoseS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\MoseS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\MoseS\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

*Improvements Made:*

Class Consolidation: Grouped rare industries (frequency < 10) into "Other".

Text Preprocessing: Added lemmatization and special character removal.

Model Upgrade: Replaced Naive Bayes with SVM (better for high-dimensional data).

TF-IDF Optimization: Increased max_features and added bigrams.